# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **data analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `dataproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from pandas_datareader import wb

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Read and clean data

In [36]:
wb_gini = wb.download(country='all', indicator='SI.POV.GINI', start=1990, end=2017)
wb_gini = wb_gini.rename(columns = {'SI.POV.GINI':'GINI'})
wb_gini = wb_gini.reset_index()
wb_gini.year = wb_gini.year.astype(int) # convert year
wb_gini.country = wb_gini.country.astype('string') # convert country to the special pandas string type
wb_gini.info()

wb_gdp = wb.download(country='all', indicator='NY.GDP.PCAP.KD', start=1990, end=2017)
wb_gdp = wb_gdp.rename(columns = {'NY.GDP.PCAP.KD':'GDP'})
wb_gdp.reset_index(inplace = True)
wb_gdp.year = wb_gdp.year.astype(int)
wb_gdp.country = wb_gdp.country.astype('string') # convert country to the special pandas string type
wb_gdp.info()
wb_gdp.head()

wb = pd.merge(wb_gini, wb_gdp, how = 'outer', on = ['country','year']);
wb.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7448 entries, 0 to 7447
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  7448 non-null   string 
 1   year     7448 non-null   int32  
 2   GINI     1582 non-null   float64
dtypes: float64(1), int32(1), string(1)
memory usage: 145.6 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7448 entries, 0 to 7447
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  7448 non-null   string 
 1   year     7448 non-null   int32  
 2   GDP      6772 non-null   float64
dtypes: float64(1), int32(1), string(1)
memory usage: 145.6 KB


,country,year,GINI,GDP
0,Africa Eastern and Southern,2017,NaN,1534.683482
1,Africa Eastern and Southern,2016,NaN,1534.924746
2,Africa Eastern and Southern,2015,NaN,1538.552268
3,Africa Eastern and Southern,2014,NaN,1537.159571
4,Africa Eastern and Southern,2013,NaN,1518.482850


In [37]:
wb_wgi=pd.read_csv("WB WGI.csv")

# These columns have to go: 'Unnamed: 0' 'Unnamed: 1' 'Unnamed: 2' 'Unnamed: 3'
drop_these = ['Country Code','Series Code'] # use list comprehension to create list of columns
wb_wgi.drop(drop_these, axis=1, inplace=True) # axis = 1 -> columns, inplace=True -> changed, no copy made
wb_wgi.head(5)

col_dict = {}
for i in range(1996, 2022): # range goes from 1996 to 2021
    col_dict[str(i)+" [YR"+str(i)+"]"] = f'wgi{i}' 
col_dict

wb_wgi.rename(columns = col_dict, inplace=True)
wb_wgi.rename (columns ={'Country Name': 'country'}, inplace=True)
wb_wgi.sample(10)

,country,Series Name,wgi1996,wgi1998,wgi2000,wgi2002,wgi2003,wgi2004,wgi2005,wgi2006,...,wgi2012,wgi2013,wgi2014,wgi2015,wgi2016,wgi2017,wgi2018,wgi2019,wgi2020,wgi2021
376,Eswatini,Rule of Law: Percentile Rank,37.1859283447266,31.5,30.348258972168,26.3681583404541,32.3383102416992,22.115385055542,22.4880390167236,33.4928245544434,...,37.5586853027344,40.3755874633789,44.230770111084,44.711540222168,38.9423065185547,45.6730766296387,45.1923065185547,35.5769233703613,33.1730766296387,31.25
620,Kuwait,Political Stability and Absence of Violence/Te...,52.6595726013184,63.2978706359863,73.0158767700195,42.3280410766602,54.773868560791,56.7961158752441,55.3398056030273,58.4541053771973,...,53.0805702209473,52.6066360473633,51.4285697937012,39.0476188659668,45.238094329834,44.761905670166,51.886791229248,54.7169799804688,54.7169799804688,55.6603775024414
691,Madagascar,Government Effectiveness: Percentile Rank,38.2513656616211,32.2404365539551,32.2404365539551,45.9459457397461,41.6216201782227,34.825870513916,42.6470603942871,31.2195129394531,...,12.7962083816528,12.3222751617432,8.17307662963867,8.17307662963867,10.0961542129517,11.057692527771,10.0961542129517,11.057692527771,13.4615383148193,13.942307472229
45,Antigua and Barbuda,Regulatory Quality: Percentile Rank,73.9130401611328,73.3695678710938,76.0869598388672,70.8108139038086,70.8108139038086,68.6567153930664,66.6666641235352,66.6666641235352,...,71.0900497436523,69.6682434082031,69.2307662963867,65.3846130371094,65.8653869628906,63.9423065185547,66.8269195556641,68.2692337036133,69.711540222168,65.8653869628906
305,Cyprus,Voice and Accountability: Percentile Rank,78.5,76.6169128417969,80.5970153808594,84.0796051025391,78.6069641113281,79.8076934814453,79.8076934814453,82.211540222168,...,79.3427200317383,77.4647903442383,78.3251266479492,80.7881774902344,82.7586212158203,83.2512283325195,80.6763305664063,81.159423828125,75.3623199462891,73.9130401611328
586,Kazakhstan,Rule of Law: Percentile Rank,14.070351600647,16,13.9303483963013,11.4427862167358,15.4228858947754,14.4230766296387,20.0956935882568,12.4401912689209,...,26.7605628967285,29.577465057373,32.211540222168,36.538459777832,33.1730766296387,33.6538467407227,34.1346168518066,33.6538467407227,36.538459777832,34.1346168518066
1082,St. Lucia,Political Stability and Absence of Violence/Te...,82.4468078613281,59.042552947998,90.476188659668,59.7883605957031,91.9598007202148,92.7184448242188,79.6116485595703,74.3961334228516,...,74.407585144043,73.459716796875,75.2380981445313,73.3333358764648,88.5714263916016,82.3809509277344,85.8490600585938,79.2452850341797,75,74.5283050537109
1216,United Kingdom,Rule of Law: Percentile Rank,94.4723587036133,95,94.5273666381836,94.0298538208008,94.0298538208008,93.2692337036133,91.8660278320313,95.6937789916992,...,94.8356781005859,93.8967132568359,94.711540222168,93.75,92.3076934814453,92.788459777832,91.8269195556641,91.8269195556641,89.9038467407227,89.4230804443359
723,Malta,Regulatory Quality: Percentile Rank,78.260871887207,83.1521759033203,84.239128112793,81.0810775756836,82.7027053833008,82.0895538330078,79.4117660522461,83.8235321044922,...,89.0995254516602,88.1516571044922,82.211540222168,84.6153869628906,85.0961532592773,87.9807662963867,88.461540222168,77.4038467407227,86.0576934814453,74.5192337036133
311,Czech Republic,Voice and Accountability: Percentile Rank,78,75.1243743896484,71.1442794799805,76.6169128417969,79.6019897460938,80.288459777832,75.961540222168,77.4038467407227,...,76.9953079223633,77.9342727661133,78.8177337646484,82.2660064697266,81.2807846069336,75.3694610595703,74.3961334228516,73.4299545288086,79.22705078125,81.159423828125


In [38]:
wb_wgi = pd.wide_to_long(wb_wgi, stubnames='wgi', i=['country','Series Name'], j='year')
wb_wgi = wb_wgi.reset_index()
# Rename colum with series names to "ser"
wb_wgi.rename(columns = {'Series Name':'ser'}, inplace=True)

#Get a list of all the series in the data
namelist = wb_wgi.ser.unique()

newnames = {'Control of Corruption: Percentile Rank':'COC',
            'Government Effectiveness: Percentile Rank':'GOV', 
            'Political Stability and Absence of Violence/Terrorism: Percentile Rank':'RSA', 
            'Regulatory Quality: Percentile Rank':'REQ', 
            'Rule of Law: Percentile Rank':'ROL', 
            'Voice and Accountability: Percentile Rank':'VOA'}

# Renames series
for index, name in enumerate(namelist):
    print(index, "Now rename Series", name, "=", newnames[name])
    
    wb_wgi.loc[wb_wgi.ser == name, 'ser'] = newnames[name]

newnamelist = wb_wgi.ser.unique()

wb_wgi.sample(5)

0 Now rename Series Control of Corruption: Percentile Rank = COC
1 Now rename Series Government Effectiveness: Percentile Rank = GOV
2 Now rename Series Political Stability and Absence of Violence/Terrorism: Percentile Rank = RSA
3 Now rename Series Regulatory Quality: Percentile Rank = REQ
4 Now rename Series Rule of Law: Percentile Rank = ROL
5 Now rename Series Voice and Accountability: Percentile Rank = VOA


,country,ser,year,wgi
29125,"Yemen, Rep.",COC,2006,23.4146347045898
7537,Dominica,REQ,2015,62.980770111084
17625,Moldova,ROL,2006,38.2775115966797
17155,Mauritius,GOV,2019,78.3653869628906
13424,Kazakhstan,GOV,2014,50


In [39]:
wb_new=pd.pivot(wb_wgi, index=['country','year'], columns = 'ser', values= 'wgi')
wb_new=wb_new.reset_index()
final = pd.merge(wb, wb_new, on=['year', 'country'], how = 'outer')

In [46]:
col_list = ['COC', 'GOV', 'REQ', 'ROL', 'RSA', 'VOA']
for i in col_list:
    final.loc[final[i]==".."] = np.nan
    final[i]=final[i].astype(float)
final.info()
final

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8494 entries, 0 to 8493
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  8191 non-null   object 
 1   year     8191 non-null   float64
 2   GINI     1577 non-null   float64
 3   GDP      6624 non-null   float64
 4   COC      4619 non-null   float64
 5   GOV      4619 non-null   float64
 6   REQ      4619 non-null   float64
 7   ROL      4619 non-null   float64
 8   RSA      4619 non-null   float64
 9   VOA      4619 non-null   float64
dtypes: float64(9), object(1)
memory usage: 730.0+ KB


,country,year,GINI,GDP,COC,GOV,REQ,ROL,RSA,VOA
0,Africa Eastern and Southern,2017.0,NaN,1534.683482,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,2016.0,NaN,1534.924746,NaN,NaN,NaN,NaN,NaN,NaN
2,Africa Eastern and Southern,2015.0,NaN,1538.552268,NaN,NaN,NaN,NaN,NaN,NaN
3,Africa Eastern and Southern,2014.0,NaN,1537.159571,NaN,NaN,NaN,NaN,NaN,NaN
4,Africa Eastern and Southern,2013.0,NaN,1518.482850,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
8489,Zambia,2021.0,NaN,NaN,25.480770,19.230770,32.211540,30.288462,48.113209,35.265701
8490,Zimbabwe,2018.0,NaN,NaN,10.096154,9.134615,5.288462,8.173077,20.283018,16.908213
8491,Zimbabwe,2019.0,NaN,NaN,10.096154,10.096154,6.250000,7.211538,14.622642,16.425121
8492,Zimbabwe,2020.0,NaN,NaN,9.615385,9.134615,7.692307,7.692307,13.207547,17.874395


In [54]:
#final.loc[final.year==".."] = np.nan
final = final.dropna(subset=['country','year'], how='any')
final.year = final.year.astype(int)
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8191 entries, 0 to 8493
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  8191 non-null   object 
 1   year     8191 non-null   int32  
 2   GINI     1577 non-null   float64
 3   GDP      6624 non-null   float64
 4   COC      4619 non-null   float64
 5   GOV      4619 non-null   float64
 6   REQ      4619 non-null   float64
 7   ROL      4619 non-null   float64
 8   RSA      4619 non-null   float64
 9   VOA      4619 non-null   float64
dtypes: float64(8), int32(1), object(1)
memory usage: 671.9+ KB


#given variabel: træk gennemsnittet fra og divider med standardafvigelsen

Import your data, either through an API or manually, and load it. 

## Explore each data set

In order to be able to **explore the raw data**, you may provide **static** and **interactive plots** to show important developments 

**Interactive plot** :

In [ ]:
def plot_func():
    # Function that operates on data set
    pass

widgets.interact(plot_func, 
    # Let the widget interact with data through plot_func()    
); 


Explain what you see when moving elements of the interactive plot around. 

# Merge data sets

Now you create combinations of your loaded data sets. Remember the illustration of a (inner) **merge**:

In [ ]:
plt.figure(figsize=(15,7))
v = venn2(subsets = (4, 4, 10), set_labels = ('Data X', 'Data Y'))
v.get_label_by_id('100').set_text('dropped')
v.get_label_by_id('010').set_text('dropped' )
v.get_label_by_id('110').set_text('included')
plt.show()

Here we are dropping elements from both data set X and data set Y. A left join would keep all observations in data X intact and subset only from Y. 

Make sure that your resulting data sets have the correct number of rows and columns. That is, be clear about which observations are thrown away. 

**Note:** Don't make Venn diagrams in your own data project. It is just for exposition. 

# Analysis

To get a quick overview of the data, we show some **summary statistics** on a meaningful aggregation. 

MAKE FURTHER ANALYSIS. EXPLAIN THE CODE BRIEFLY AND SUMMARIZE THE RESULTS.

# Conclusion

ADD CONCISE CONLUSION.